In [1]:
first_question_article = [
     {'article_id': '00000194-25d6-dcc4-a1d7-3df6703c0000',
      'question': 'אני רוצה לקרוא ביקורת על הסרט החדש של לוקה גואדנינו, זה שמבוסס על ספר של בורוז על הומו שחי במקסיקו בשנות ה-50 ומחפש גברים צעירים.'},
      {'article_id': '00000194-2618-dd68-a3be-e6fc06680000',
       'question': 'אני מחפש ביקורת על הסרט החדש עם ניקול קידמן, זה שהיא משחקת בו מנכ"לית חברת רובוטיקה שיש לה רומן עם מתמחה צעיר. הסרט הוקרן בפסטיבל ונציה, איך הוא?.'
       },
       {'article_id': '00000194-2681-ddb6-afdd-77e781220000',
        'question': 'אני מחפש ביקורת על הסרט החדש של עמוס גיתאי, זה שעוסק בשאלה למה יש מלחמות ומשתמש במכתבים של איינשטיין ופרויד.'
        },
        {'article_id': '00000194-2b97-‎d9c2-a79e-2bd7330c0000',
            'question': 'אפשר בבקשה ביקורת על הסרט החדש "נוספרטו" של רוברט אגרס? אני רוצה לדעת אם הוא נאמן למקורות של סרטי הערפדים הקלאסיים ואיך השחקנים הראשיים, במיוחד לילי-רוז דפ, משחקים?'
        },
        {'article_id': '00000194-319b-d555-abbc-b1dfdf250000',
            'question': '"אני מחפש ביקורת על סרט חדש שמוקרן בפסטיבל חיפה, על אישה אלכוהוליסטית בשם רונה שחוזרת לאי הולדתה בסקוטלנד כדי להתמודד עם ההתמכרות שלה. מישהי כתבה על זה ביקורת?'
        },
        {
            'article_id': '00000194-31db-ddaf-adb7-7bfbaf200000',
            'question': 'אני מחפש סרט איראני חדש שמתרחש בטהרן ועוסק במשפחה שאבא שלה שופט ורואים בו גם קטעים מההפגנות שם. ראיתי שהקרינו אותו בפסטיבל חיפה'
        },
        {'article_id': '00000194-34df-d39d-a196-b7ff86ce0000',
            'question': '"אני מחפש סדרת אנימציה חדשה בנטפליקס, משהו בסגנון באפי קוטלת הערפדים אבל עם מיתולוגיה סינית. שמעתי שיש סדרה על נערה סינית-אמריקאית שנלחמת בשדים, מישהו מכיר?',
        },
        {
            'article_id':"00000194-356b-de88-a3dc-75ef26460000",
            'question':"אוקיי, שמעתי על סרט חדש של שבי גביזון עם ריצ'רד גיר על אבא שמגלה שהיה לו בן שנהרג. איך קוראים לסרט ומה הוא מספר עליו?"
        },
        {
        'article_id':'00000194-4be5-dff1-a7bc-dbf57a210000',
        'question':'אני מחפש סרט על שני בני דודים יהודים אמריקאים שנוסעים לפולין בעקבות השואה. אחד מהם משחק בסדרה "יורשים". מישהו כתב וביים את הסרט והוא גם שחקן בסרט "הרשת החברתית".',
        },
        {
        'article_id':'00000194-6339-dabc-afff-6b7fb6e90000',
        'question':'אוקיי, שמעתי שיש סדרה חדשה בנטפליקס, מין מערבון כזה אבל ממש קשוח, עם הרבה אלימות וטבח. זה על איזה אישה שמנסה לשרוד עם הבן שלה בתקופה של מלחמת יוטה והטבח בהר מדוז. מישהו יודע על מה אני מדבר?',
        },
        {
        'article_id':'00000194-6e60-d9d3-a597-eeecd15d0000',
        'question':'"אני מחפש מידע על הסרט החדש של ארז תדמור, זה שמתרחש בשנות ה-50 ליד הכנרת ועוסק בפרטיזנים לשעבר שעובדים במפעל סודה. איך קוראים לסרט ומה העלילה שלו?',
        },
]

first_question_genre = [
    "אחי, תמליץ לי על איזה סרט אקשן טוב בנטפליקס? משהו חדש יחסית.",
    "מחפשת סדרה קלילה ומצחיקה, קומדיה כזאת, לא משנה לי באיזה שירות סטרימינג.",
    "מה כדאי לראות עכשיו ביס פלוס? בא לי איזה משהו מתח טוב.",
    "יש לך המלצה לסרט דרמה איכותי? עדיף משהו שזמין בהוט.",
    "אני והחבר'ה רוצים לראות סרט אימה הערב. יש משהו ממש מפחיד שאתה מכיר?",
    "תגיד, יש איזה סדרת מדע בדיוני שווה בפרטנר TV או בסלקום TV?",
    "בא לי לראות סרט ישראלי טוב, יש משהו מומלץ לאחרונה?",
    "מחפש סדרה לבינג', משהו שאפשר לטחון בסופ\"ש. ז'אנר פחות קריטי, העיקר שתהיה ממכרת.",
    "יש סרטי דוקו מעניינים בדיסני פלוס או באפל טי וי?",
    "סתם תן המלצה לסרט טוב שראית לאחרונה, לא משנה לי הז'אנר או הפלטפורמה."

]

genre_questions_as_dicts = [
    {'article_id': '', 'question': q} for q in first_question_genre
]

# איחוד שתי הרשימות לרשימה אחת
combined_questions = first_question_article + genre_questions_as_dicts

second_question = 'איזה יצירה המבקר העדיף יותר?'
thierd_question = 'אם הייתי אומר לך שאני במיוחד רגיש לסיומים מאכזבים/לא סגורים, איזו יצרה עדיפה מבין אלו שהמלצת?'



In [3]:
from src.llm_api_client import LLMClient
from config.load_config import load_config

from test_antrpoic.antropic import ChatBot
import streamlit as st


prompts = load_config("config/prompts.yaml")
sys_instruct = prompts.get("system_instructions")
config = load_config("config/config.yaml")
llm_config = config.get("llm", {})
api_key = llm_config.get("GOOGLE_API_KEY")
model_name = llm_config.get("llm_model_name")

result = []

import numpy as np
counter = np.random.randint(0, 9999999999)

# Initialize the LLM client gemini
llm_client = LLMClient(model_name=model_name, api_key=api_key, sys_instruct=sys_instruct, config=config)

# Initialize the LLM client antropic
antrp = ChatBot(config)



for q in combined_questions:
    st.session_state.messages = []
    counter += 1


    print('\n')
    print("="*20)
    article_id = q.get('article_id')
    question = q.get('question')
    print(f"Article ID: {article_id}")
    print(f"Question: {question}")
    
    full_response = ""
    ans = llm_client.streaming_message(question,f'{counter}')
    async for chunk in ans:
        print(chunk, end='', flush=True)
        if chunk:
            full_response += chunk


    print('\n')
    print("="*20)
    second_answer = ""
    ans = llm_client.streaming_message(second_question,f'{counter}')
    async for chunk in ans:
        if chunk:
            second_answer += chunk
        print(chunk, end='', flush=True)


    thierd_answer = ""
    ans = llm_client.streaming_message(thierd_question,f'{counter}')
    async for chunk in ans:
        if chunk:
            thierd_answer += chunk
        print(chunk, end='', flush=True)



    print('antrpoic answer')
    antropic_answer, state = antrp.process_user_input(question,st.session_state)
    print(antropic_answer)
    second_answer_antrpoic, state = antrp.process_user_input(second_question,state)
    print(second_answer_antrpoic)
    thierd_answer_antrpoic, state = antrp.process_user_input(thierd_question,state)
    print(thierd_answer_antrpoic)

    result.append({
        'article_id': article_id,
        'question': question,
        'answer_gemini': full_response,
        'answer_antrp': antropic_answer,
        'second_answer_gemini': second_answer,
        'second_answer_antrp': second_answer_antrpoic,
        'thierd_answer_gemini': thierd_answer,
        'thierd_answer_antrp': thierd_answer_antrpoic
    })
    
    


{"message": "Successfully connected to Redis."}
2025-04-23 18:59:02.559 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-23 18:59:02.562 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
{"message": "No history found for user 5833732142 in Redis."}
{"message": "Successfully created new chat session."}




Article ID: 00000194-25d6-dcc4-a1d7-3df6703c0000
Question: אני רוצה לקרוא ביקורת על הסרט החדש של לוקה גואדנינו, זה שמבוסס על ספר של בורוז על הומו שחי במקסיקו בשנות ה-50 ומחפש גברים צעירים.
אהלן! בטח, אני אשמח לעזור לך למצוא את הביקורת שאתה מחפש על הסרט החדש של לוקה גואדנינו, זה שמבוסס על ספר של בורוז ומתרחש במקסיקו של שנות ה-50. זה נשמע כמו נושא מרתק ומורכב. אני כבר מחפש לך את המידע והביקורת הכי מעניינים שיש לנו. חכה רגע ואני אמצא לך את זה!


{"message": "Processing function call: find_movie_tv_info_or_review with args: {'query': '\u05d1\u05d9\u05e7\u05d5\u05e8\u05ea \u05e2\u05dc \u05d4\u05e1\u05e8\u05d8 \u05d4\u05d7\u05d3\u05e9 \u05e9\u05dc \u05dc\u05d5\u05e7\u05d4 \u05d2\u05d5\u05d0\u05d3\u05e0\u05d9\u05e0\u05d5, \u05d6\u05d4 \u05e9\u05de\u05d1\u05d5\u05e1\u05e1 \u05e2\u05dc \u05e1\u05e4\u05e8 \u05e9\u05dc \u05d1\u05d5\u05e8\u05d5\u05d6 \u05e2\u05dc \u05d4\u05d5\u05de\u05d5 \u05e9\u05d7\u05d9 \u05d1\u05de\u05e7\u05e1\u05d9\u05e7\u05d5 \u05d1\u05e9\u05e0\u05d5\u05ea \u05d4-50 \u05d5\u05de\u05d7\u05e4\u05e9 \u05d2\u05d1\u05e8\u05d9\u05dd \u05e6\u05e2\u05d9\u05e8\u05d9\u05dd.'}"}
{"message": "Query is likely Hebrew, returning original query."}
{"message": "Retrieving relevant documents for query: '\u05d1\u05d9\u05e7\u05d5\u05e8\u05ea \u05e2\u05dc \u05d4\u05e1\u05e8\u05d8 \u05d4\u05d7\u05d3\u05e9 \u05e9\u05dc \u05dc\u05d5\u05e7\u05d4 \u05d2\u05d5\u05d0\u05d3\u05e0\u05d9\u05e0\u05d5, \u05d6\u05d4 \u05e9\u05de\u05d1\u05d5\u05e1

[{"article_id": "00000194-25d6-dcc4-a1d7-3df6703c0000", "article_name": "\"קוויר\": מרתק לראות את דניאל קרייג מפגין שילוב של חולשה וגבריות יהירה", "writer_name": ["פבלו אוטין"]}, {"article_id": "0000017f-e3db-df7c-a5ff-e3fbc9000000", "article_name": "אז איך זה נראה כשהוליווד המיינסטרימית עוסקת סוף סוף במתבגרים הומואים?", "writer_name": ["אורי קליין"]}]מצוין! מצאתי עבורך ביקורת מעולה על הסרט שחיפשת – "קוויר" (Queer) של לוקה גואדנינו. נשמע שזה בדיוק הסרט המורכב והמעניין שתיארת.

הביקורת, שנכתבה על ידי פבלו אוטין, מדברת בדיוק על האדפטציה הזו של גואדנינו לספר "קוויר" של ויליאם ס. בורוז, שמתרחשת במקסיקו של שנות ה-50 ועוקבת אחרי ויליאם לי שמחפש גברים צעירים.

הכותב מציין שהסרט לא רק מעבד את הסיפור המקורי אלא גם מהווה מעין ביקורת ספרותית עליו, כשהוא משלב אלמנטים סוריאליסטיים ואת ההקשרים סביב כתיבת הספר המקורי. הוא מתאר את החוויה הקולנועית כ"חוויה תודעתית ומשונה".

דניאל קרייג, שמגלם את ויליאם לי, מקבל שבחים על תפקידו. הביקורת מציינת ש"מרתק לראות את קרייג מפגין שילוב של פגיעות וחולשה עם גבריות

{"message": "Loaded history for user 5833732142 from Redis."}
{"message": "Successfully created new chat session."}


...

מצאתי בשבילך שתי המלצות מעניינות לסרטים חדשים שקיבלו ביקורות טובות:

1. "ברברי" (Barbarian) - סרט אימה מפתיע ומדובר שזכה להצלחה גדולה בארה"ב. הסרט משלב אלמנטים של מתח ואימה עם תובנות חברתיות מעניינות. זהו סרט שמשחק עם ציפיות הצופים ומציע חוויה קולנועית מרתקת ולא שגרתית. אפשר לצפות בו בדיסני+.

2. "צ'ה צ'ה ממש חלק" (Cha Cha Real Smooth) - סרט אינדי מקסים שזכה בפרס חביב הקהל בפסטיבל סאנדנס. זוהי דרמה קומית חמה ולבבית על צעיר בן 25 שמוצא את דרכו בחיים דרך עבודה כמנחה במסיבות בר מצווה. הסרט מציע תמהיל מוצלח של הומור ורגש, עם משחק משובח של דקוטה ג'ונסון. זמין לצפייה באפל טיוי+.

אם אתה מחפש משהו מרענן ולא שגרתי, שני הסרטים האלה יכולים להתאים. "ברברי" הוא בחירה טובה אם אתה אוהב סרטי אימה מתוחכמים, ו"צ'ה צ'ה ממש חלק" מתאים יותר אם אתה מחפש סרט קליל ונעים עם נגיעות של עומק רגשי.


2025-04-23 19:23:47.391 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-23 19:23:47.392 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-23 19:23:47.393 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


<thinking>
אקרא בעיון את שתי הביקורות כדי להשוות את התייחסות המבקר לשני הסרטים ולזהות איזה מהם קיבל ביקורת חיובית יותר.
</thinking>

לאחר קריאה מעמיקה של שתי הביקורות, נראה שהמבקר העדיף את "צ'ה צ'ה ממש חלק". הנה למה:

בעוד שעל "ברברי" המבקר היה מעורב יותר בדעתו, וציין שהסיום היה "החלק החלש ביותר בסרט" והעיר שהסרט "אינו מצמרר באמת או מחדש", לגבי "צ'ה צ'ה ממש חלק" הביקורת הייתה חיובית באופן עקבי יותר. המבקר משבח את הסרט על היותו "יצירה לבבית וחיננית", מציין את "נוכחות המסך" המרשימה של היוצר, ומסיים את הביקורת באמירה חיובית: "נסו גם אתם לעמוד בפני רגישותו וטוּב הלב המשתפך שלו – ותשמחו להיכשל."

המבקר אף ממליץ למעשה על "ברברי" בהסתייגות: "אולי בכל זאת עדיף סתם להיתקל ב'ברברי' בזמן שיטוט בתפריט של שירות סטרימינג ולא להגיע אליו במיוחד ועם ציפיות בשמים."


2025-04-23 19:23:56.277 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


<thinking>
אנתח את ההתייחסות לסיומים בשתי הביקורות, במיוחד לאור הרגישות המוצהרת של המשתמש לסיומים לא מספקים.
</thinking>

בהתחשב ברגישותך לסיומים מאכזבים, הייתי ממליץ על "צ'ה צ'ה ממש חלק". הנה למה:

לגבי "ברברי", המבקר מציין במפורש בעיות עם הסיום:
- "בשבילי הסיום היה החלק החלש ביותר בסרט"
- הסרט מוביל ל"אחת משתי אפשרויות — סיום הולם ומספק, או אכזבה רבתי"
- המבקר מציין שהסרט "החל לאבד אחיזה" לקראת הסיום

לעומת זאת, ב"צ'ה צ'ה ממש חלק" אין התייחסות שלילית לסיום הסרט. הביקורת מתארת אותו כסרט שלם ועקבי לכל אורכו, עם דגש על חוויה רגשית מספקת.

לכן, אם אתה רגיש במיוחד לסיומים מאכזבים, "צ'ה צ'ה ממש חלק" יהיה בחירה בטוחה יותר עבורך.


In [4]:
import pandas as pd
pd.DataFrame(result).to_excel('first_question_article.xlsx', index=False)

In [ ]:
second_answer

'אחלה שאלה! כדי לענות לך, אני צריך לדעת על אילו יצירות מדובר. ברגע שתגיד לי את השמות שלהן, או אפילו תיתן לי קצת פרטים עליהן, אוכל לצלול לביקורות ולראות איזו מהן המבקר העדיף ולמה. יאללה, ספר לי! 😊'

In [ ]:
full_response

'אהלן! וואו, יש כל כך הרבה דברים טובים לראות עכשיו... בוא נראה איזה סרט יוכל להתאים לך. אני אבדוק שניה ואחזור אליך עם משהו שווה במיוחד!'

In [ ]:
thierd_answer

'הבנתי אותך לגמרי! אין מצב שנתקע עם סוף שישאיר אותך עם טעם מר או מלא סימני שאלה.\n\nהעניין הוא שעוד לא המלצתי לך על יצירות ספציפיות בשיחה הזו. כדי שאוכל להשוות ביניהן ולהגיד לך איזו מהן עדיפה מבחינת הסוף, אני צריך לדעת על אילו יצירות מדובר.\n\nספר לי אילו סרטים או סדרות עומדות על הפרק מבחינתך, ואני מיד אצלול לפרטים, אבדוק את עניין הסוף (בלי ספוילרים כמובן!), ואגיד לך איזו מהן תתאים יותר לרגישות שלך לסופים מאכזבים.\n\nאו שאפשר פשוט שאחפש לך עכשיו המלצה על יצירה עם סוף טוב וסגור במיוחד? מה אתה מעדיף? 😊'